# Database construction on Sqlite3

Firstly, we need to save all the needed PMIDs from PubMed, which can be easily done by clicking "save" button on PubMed and filter the results.

An example of the saved text file

In [1]:
path = 'pmid-HeartTrans_mortality-set.txt'
with open(path,'r',encoding='utf-8') as f:
    pmid_HtM = f.read()
pmid_HtM_list = pmid_HtM.split("\n")
pmid_HtM_list[0]

'28413189'

Insert all the articles into the table

In [76]:
## Run once only
import gzip
import json
import time
import sqlite3

def jsonlreader(path):
    for line in gzip.open(path, "rt"):
        yield json.loads(line)

conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')        
cu = conn.cursor()
## create table
# cu.execute('''create table if not exists articles(
#         pmid TEXT,
#         title TEXT,
#         abstract TEXT,
#         mesh TEXT,
#         keywords TEXT,
#         pubyear INTEGER)''')

i = 0
articles_list = []
start = time.time()
for article in jsonlreader("D:\\pubmed22.jsonl.gz"):
    articles_list.append((article["PMID"],article["Title"],str(article["Abstract"]),str(article["MeSH"])\
                          ,str(article["Keywords"]),article["PubDate"]["Year"]))
    i += 1
    if i % 500000 == 0 or i == 33405863:
        cu.executemany('INSERT INTO articles VALUES (?,?,?,?,?,?);',articles_list)
        articles_list = []
        print('We have inserted', cu.rowcount, 'articles to the table.') 
end = time.time()
print("Time cost:",end - start)
conn.commit()
conn.close()

We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inserted 500000 articles to the table.
We have inser

Add index to the table to make it searching faster

In [78]:
## Run once only
## create index for pmid 
start = time.time()
conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')
c = conn.cursor()
c.execute("CREATE INDEX pmid_index ON articles(pmid);") # create index
# c.execute("DROP INDEX pmid_index;") # delete index
# c.execute("DELETE from articles;") # delete table
conn.commit()
conn.close()
end = time.time()
print("Time cost:",end - start)

Time cost: 405.51740169525146


### Construct the positive dataset

In [2]:
import time
import sqlite3
import ast
import os

Check if there are duplicated PMIDs in the saved text file

In [3]:
## Positive pmid
path = "pmid-HeartTrans_Mortality-set.txt"
with open(path,'r',encoding='utf-8') as f:
    text = f.read()
positive = text.split("\n")
p = set(positive)
if len(positive) == len(p):
    print("No duplicated PMID in the list.")
else:
    le = len(positive) - len(p)
    print("%d duplicated PMIDs in the list have been removed, and the order in the list was not changed." % le)
    new_list = list(p)
    new_list.sort(key=positive.index)
    positive = new_list

No duplicated PMID in the list.


Check if the corresponding articles have "Mortality"

In [4]:
## Test positive data
conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')
cu = conn.cursor()
statement = "SELECT * from articles  WHERE pmid in ({0})".format(', '.join(['?'] * len(positive)))
article = cu.execute(statement, positive)
i = 0
for row in article:
    s = ""
    pmid = row[0]
    title = row[1]
    abstact = row[2]
    mesh = row[3]
    keywords = row[4]
    mesh_list = ast.literal_eval(mesh)
    key = 0
    for term in mesh_list:
        if term["Name"] in ["Heart Transplantation","Heart-Lung Transplantation"]:
            for qual in term["Qualifiers"]:
                if qual["Name"] == "mortality":
                    key = 1
    if not key:
        print(pmid)
print("the articles above don't have the qualifier 'Mortality' in database but they indeed are tagged with it on PubMed.")
#         break

32798029
32928549
33074524
33405353
33535932
33632637
33675715
33745714
33955021
33965332
34045062
34243724
34585490
34689572
34752418
34820983
the articles above don't have the qualifier 'Mortality' in database but they indeed are tagged with it on PubMed.


Check if the corresponding abstracts exist in the database, skip them and save the relevant abstracts in text.

In [5]:
## Extract information and consturct the positive dataset
start = time.time()
conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')
cu = conn.cursor()
statement = "SELECT PMID, Title, Abstract, Keywords from articles  WHERE pmid in ({0})".format(', '.join(['?'] * len(positive)))
article = cu.execute(statement, positive)
i = 0
positive_noabs = []
for row in article:
    s = ""
    pmid = row[0]
    title = row[1]
    abstact = row[2]
    keywords = row[3]
    abstract_dic = ast.literal_eval(abstact)
    
    for subt in abstract_dic:
        s1 = subt["Text"]
        if s1 == None:
            continue
        else:
            if s == "":
                s = s + s1 
            else:
                s = s + " " + s1
        s1 = ""
        
    if s == "":
        print("article without the abstract, the pmid is %s" % pmid)
        positive.remove(pmid)
        positive_noabs.append(pmid) 
        continue        
        
    keywords = keywords.strip("[]")
    text = title + " " + s + " " + keywords
    
    if i<1000:
        p = 'training dataset/positive/%s.txt' % pmid
    elif i<1500:
        p = 'validation dataset/positive/%s.txt' % pmid
    else:
        p = 'test dataset/positive/%s.txt' % pmid
        
#     f=open(p, "w",encoding='utf-8')
#     f.write(text)
#     f.close()
    i += 1
       
print("There are %d positive articles in total" % (i))
conn.close()
end = time.time()
print("Time cost:",end - start)

article without the abstract, the pmid is 28110488
article without the abstract, the pmid is 30821595
There are 1746 positive articles in total
Time cost: 0.21293163299560547


### Construct the negative dataset

In [6]:
## Negative pmid
path = "pmid-HeartTrans-set.txt"
with open(path,'r',encoding='utf-8') as f:
    text = f.read()
negative = text.split("\n")
n = set(negative)
if len(negative) == len(n):
    print("No duplicated PMID in the list.")
else:
    le = len(negative) - len(n)
    print("%d duplicated PMIDs in the list have been removed, and the order in the list was not changed." % le)
    new_list = list(n)
    new_list.sort(key=negative.index)
    negative = new_list


116 duplicated PMIDs in the list have been removed, and the order in the list was not changed.


In [7]:
i = 0
for pos in positive:
    if pos in negative:
        negative.remove(pos)
        i += 1
for pos in positive_noabs:
    if pos in negative:
        negative.remove(pos)
        i += 1
print("%d positive articles have been removed from the negative dataset" % i)
print("Now there are %d articles in the negative dataset" % len(negative))

1748 positive articles have been removed from the negative dataset
Now there are 15273 articles in the negative dataset


In [8]:
## Test negative data
conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')
cu = conn.cursor()
statement = "SELECT * from articles  WHERE pmid in ({0})".format(', '.join(['?'] * len(negative)))
article = cu.execute(statement, negative)
i = 0
for row in article:
    s = ""
    pmid = row[0]
    title = row[1]
    abstact = row[2]
    mesh = row[3]
    keywords = row[4]
    mesh_list = ast.literal_eval(mesh)
    key = 0
    for term in mesh_list:
        if term["Name"] in ["Heart Transplantation","Heart-Lung Transplantation"]:
            for qual in term["Qualifiers"]:
                if qual["Name"] == "mortality":
                    key = 1
    if key:
        print("positive article in the negative dataset!!! " + pmid)
    
    key = 0
    for term in mesh_list:
        if term["Name"] in ["Heart Transplantation","Heart-Lung Transplantation"]:
                key = 1  
    if not key:
        if mesh == "[]":
            print("article without mesh term heart_tran " + pmid)
        else:
            print("article error " + pmid)

article without mesh term heart_tran 31891373
article without mesh term heart_tran 31939750
article without mesh term heart_tran 31993956
article without mesh term heart_tran 32354629
article without mesh term heart_tran 32372168
article without mesh term heart_tran 32462258
article without mesh term heart_tran 32572551
article without mesh term heart_tran 32703639
article without mesh term heart_tran 32739334
article without mesh term heart_tran 32768298
article without mesh term heart_tran 32773322
article without mesh term heart_tran 32778365
article without mesh term heart_tran 32841466
article without mesh term heart_tran 32858640
article without mesh term heart_tran 32861553
article without mesh term heart_tran 32870436
article without mesh term heart_tran 32889634
article without mesh term heart_tran 32942278
article without mesh term heart_tran 32950380
article without mesh term heart_tran 32981709
article without mesh term heart_tran 33002249
article without mesh term heart_tr

In [9]:
## Extract information and consturct the negative dataset
negative_noabs = []
start = time.time()
conn = sqlite3.connect('D:\\PubMed_Sqlite3\\pubmed.db')
cu = conn.cursor()
statement = "SELECT PMID, Title, Abstract, Keywords from articles  WHERE pmid in ({0})".format(', '.join(['?'] * len(negative)))
article = cu.execute(statement, negative)
i = 0
for row in article:
    s = ""
    pmid = row[0]
    title = row[1]
    abstact = row[2]
    keywords = row[3]
    abstract_dic = ast.literal_eval(abstact)
    
    for subt in abstract_dic:
        s1 = subt["Text"]
        if s1 == None:
            continue

        else:
            if s == "":
                s = s + s1 
            else:
                s = s + " " + s1
        s1 = ""
    
    if s == "":
        print("article without the abstract, the pmid is %s" % pmid)
        negative_noabs.append(pmid)
        negative.remove(pmid)
        continue
        
    keywords = keywords.strip("[]")
    try:
        text = title + " " + s + " " + keywords
    except:
        text = s + " " + keywords
    
    if i<1000:
        p = 'training dataset/negative/%s.txt' % pmid
    elif i<1500:
        p = 'validation dataset/negative/%s.txt' % pmid
    else:
        p = 'test dataset/negative/%s.txt' % pmid
        
#     f=open(p, "w",encoding='utf-8')
#     f.write(text)
#     f.close()
    i += 1
       
print("There are %d negative articles in total" % (i+1))
conn.close()
end = time.time()
print("Time cost:",end - start)

article without the abstract, the pmid is 19876441
article without the abstract, the pmid is 20385962
article without the abstract, the pmid is 23304053
article without the abstract, the pmid is 25002571
article without the abstract, the pmid is 26216213
article without the abstract, the pmid is 26655937
article without the abstract, the pmid is 27903010
article without the abstract, the pmid is 28030875
article without the abstract, the pmid is 28316176
article without the abstract, the pmid is 28545281
article without the abstract, the pmid is 29799959
article without the abstract, the pmid is 30096261
article without the abstract, the pmid is 30694075
article without the abstract, the pmid is 31132883
article without the abstract, the pmid is 31315301
article without the abstract, the pmid is 31431128
article without the abstract, the pmid is 31584336
article without the abstract, the pmid is 31661343
article without the abstract, the pmid is 31672395
article without the abstract, t